In [4]:
import os
from tokenize import Number
import pandas as pd
import numpy as np
import re
from datetime import date, datetime, timedelta
from faker import Faker    
fake = Faker()  
from pymongo import MongoClient

In [6]:
from pandas.core.frame import DataFrame
pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
def read_text_file(file_path,rows=0):
    rowNumber = extractWeekNumber(file_path)
    data = pd.read_excel(file_path,skiprows=[i for i in range(0,rows-1)])
    if (rowNumber):
        data['Semana'] = rowNumber
    #print(data.head)
    return data


In [10]:
def extractWeekNumber(path):
    match = re.search(r"semana_(\d+)", path)
    if match:
        week_number = match.group(1)
        return week_number
    else:
        print("No week number found in the file path.")

In [ ]:
def main(path):
    print(path)
    data = []
    os.chdir(path)
    files = ""
    for file in os.listdir():
        if file.endswith(".xlsx"):
            file_path = f"{path}\{file}"
            if "semana" in file_path.lower():
                print(f"Processing {file_path}...")
                data.append(read_text_file(file_path))
                print(f"Processed {file_path}")
    if len(data) != 0:
        data = pd.concat(data)
    else:
        print('Error, debe tener el archivo datos en xlsx y archivo ecat en txt')

    return data

<string>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\{'
<string>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\{'
C:\Users\gerda\AppData\Local\Temp\ipykernel_2408\3751773222.py:8: SyntaxWarning: invalid escape sequence '\{'
  file_path = f"{path}\{file}"


In [100]:
def generateTrainingFiles(df):   
    fields = ['Fecha', 'productId', 'storeId', 'País', 'Venta dia anterior',
       'Stock dia actual', 'Last 4 Weeks Avg', 'Last Sale Date',
       'Days Since Last Sale', 'Total Presence', 'total tiendas',
        '% global tiendas', 'Percentage Difference',
       'Condition', 'Remaining Days', 'Remaining Broke', 'has sales',
       'enough information', 'category']
    
    uniqueDate = np.sort(df['Fecha'].unique())    
    previous_data = readResults()
    for date in uniqueDate:
        print(date)
        df_date = df[df['Fecha'] == date]            
        df_date['Código Barra CP'] = (df_date['Código Barra CP']).astype(str)  
        joined_df = preproccingDF(df_date)
        
        if not previous_data.empty:
            joined_df = pd.concat([joined_df, previous_data], ignore_index=True)
            
        df_test = joined_df.copy()
        df_test = calculateDF(df_test)
        df_test['productId'] = (df_test['productId']).astype(str)  

        formatted_date = pd.Timestamp(date).strftime("%Y_%m_%d")
        processed_file_name = f"processed_{formatted_date}.xlsx"   
        df_test['Last Sale Date'] = df_test['Last Sale Date'].apply(lambda x: None if pd.isna(x) or x == "NaT" else x)  # Replace NaT with None
        df_test['Fecha'] = df_test['Fecha'].apply(lambda x: None if pd.isna(x) else x)
        insertResults(df_test[df_test['Fecha'] == date])
        previous_data = pd.concat([previous_data, df_test[df_test['Fecha'] == date][fields]], ignore_index=True)
        #df_test.to_excel(processed_file_name,index=False)        

# Procesamiento

In [17]:
catalogDescription = read_text_file("C:/Users/gerda/Documentos_NoDrive/Maestria/2024/4to Trimestre/Product Developtment/Proyecto final/retail_classificator/analyisis/products.xlsx")  
storeDescription = read_text_file("C:/Users/gerda/Documentos_NoDrive/Maestria/2024/4to Trimestre/Product Developtment/Proyecto final/retail_classificator/analyisis/stores.xlsx")  
categoryDescription = read_text_file("C:/Users/gerda/Documentos_NoDrive/Maestria/2024/4to Trimestre/Product Developtment/Proyecto final/retail_classificator/analyisis/categories.xlsx")
catalogDescription['productId'] = (catalogDescription['productId']).astype(str)  
catalogDescription['OriginalBarCode'] = (catalogDescription['OriginalBarCode']).astype(str)  
catalogDescription['OriginalBarCode'] = catalogDescription['OriginalBarCode'].apply(lambda x: x.rstrip(".0") if x.endswith(".0") else x)
catalogDescription

No week number found in the file path.
No week number found in the file path.
No week number found in the file path.


,productDescription,productId,OriginalBarCode
0,mesh cross-media paradigms,1086152146635,9917613372
1,redefine web-enabled bandwidth,5657709229802,750954665591
2,extend intuitive partnerships,8890014708724,750954605515
3,synthesize dynamic action-items,9940478863107,9917600352
4,orchestrate killer mindshare,3773144248672,750954600312
...,...,...,...
1995,target magnetic info-mediaries,9607010680133,nan
1996,reinvent open-source networks,7402124698188,nan
1997,syndicate B2B solutions,2584583409989,nan
1998,exploit cross-media e-business,1148257983796,nan


In [126]:
#path = f"{os.getcwd()}/data/dummy"
data = main(path)
data

C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy
Processing C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy\walmart_semana_21_d.xlsx...
Processed C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy\walmart_semana_21_d.xlsx
Processing C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy\walmart_semana_21_s.xlsx...
Processed C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy\walmart_semana_21_s.xlsx
Processing C:\Users\gerda\Documentos_NoDrive\Maestria\2024\4to Trimestre\Product Developtment\Proyecto final\retail_classificator\analyisis/data/dummy\w

,Item Nbr Cliente,Signing Desc,Store Nbr,Store Name,Inv On Hand,Inv en Tránsito,Inv preparandose en CD,UxC,Dept Category Description (cadena),Price,Promedio Rotación Semanal,País,Código Barra CP,Cadena,SKU ABC CP,Fecha,SubBrand CP,Semana
0,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3700,WM SAN SEBASTIAN,114,360,0,24,CLEANERS - HOUSEHOLD D13,31.723011,34,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
1,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3701,WM HEREDIA,100,144,0,24,CLEANERS - HOUSEHOLD D13,30.789981,33,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
2,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3702,WM CURRIDABAT,312,216,0,24,CLEANERS - HOUSEHOLD D13,49.450575,53,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
3,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3705,WM GUADALUPE,95,72,72,24,CLEANERS - HOUSEHOLD D13,26.124832,28,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
4,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM4006,WM CARTAGO,190,24,0,24,CLEANERS - HOUSEHOLD D13,20.526654,22,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM734,MD ILOBASCO,18,0,0,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128971,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM783,MD AHUACHAPAN,0,0,36,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128972,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM787,MD SAN MIGUEL PANAMERICAN,18,0,0,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128973,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM798,MD SAN JUAN,0,0,18,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25


In [128]:
data.columns

Index(['Item Nbr Cliente', 'Signing Desc', 'Store Nbr', 'Store Name',
       'Inv On Hand', 'Inv en Tránsito', 'Inv preparandose en CD', 'UxC',
       'Dept Category Description (cadena)', 'Price',
       'Promedio Rotación Semanal', 'País', 'Código Barra CP', 'Cadena',
       'SKU ABC CP', 'Fecha', 'SubBrand CP', 'Semana'],
      dtype='object')

In [130]:
df_training = data.copy()
df_training

,Item Nbr Cliente,Signing Desc,Store Nbr,Store Name,Inv On Hand,Inv en Tránsito,Inv preparandose en CD,UxC,Dept Category Description (cadena),Price,Promedio Rotación Semanal,País,Código Barra CP,Cadena,SKU ABC CP,Fecha,SubBrand CP,Semana
0,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3700,WM SAN SEBASTIAN,114,360,0,24,CLEANERS - HOUSEHOLD D13,31.723011,34,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
1,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3701,WM HEREDIA,100,144,0,24,CLEANERS - HOUSEHOLD D13,30.789981,33,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
2,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3702,WM CURRIDABAT,312,216,0,24,CLEANERS - HOUSEHOLD D13,49.450575,53,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
3,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM3705,WM GUADALUPE,95,72,72,24,CLEANERS - HOUSEHOLD D13,26.124832,28,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
4,1306419,LIMP PULIDOR AJAX C TRICL 250GR,CRWM4006,WM CARTAGO,190,24,0,24,CLEANERS - HOUSEHOLD D13,20.526654,22,CR,9917612088,WALMART,C,2023-05-27,CLEANERS - HOUSEHOLD D13,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM734,MD ILOBASCO,18,0,0,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128971,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM783,MD AHUACHAPAN,0,0,36,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128972,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM787,MD SAN MIGUEL PANAMERICAN,18,0,0,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25
128973,75027516,FC SUAVITEL C. SUPERIOR PRIMAVERAL 600ML,SVWM798,MD SAN JUAN,0,0,18,18,LAUNDRY SOFTENER-FRESHENER D13,0.000000,0,SV,750954668931,WALMART,C,2023-06-23,LAUNDRY SOFTENER-FRESHENER D13,25


In [132]:
generateTrainingFiles(df_training)

Total of: (2901445, 19)
2023-05-26T00:00:00.000000000
Duplicate rows found:
(12149, 12)
Duplicates resolved. Cleaned data:
(117243, 6)
Execute calculate_last_4_weeks_avg_optimized
Execute calculate_oldest_sale_date_vectorized
Execute totalPresence
Execute defineRotationType
Execute calculate_percentage_diff_and_condition
Execute calculate_remaining_days_and_broke
Execute checkSales
Fecha
Last Sale Date
Data successfully inserted into MongoDB!
2023-05-27T00:00:00.000000000
Duplicate rows found:
(12153, 12)
Duplicates resolved. Cleaned data:
(117580, 6)
Execute calculate_last_4_weeks_avg_optimized
Execute calculate_oldest_sale_date_vectorized
Execute totalPresence
Execute defineRotationType
Execute calculate_percentage_diff_and_condition
Execute calculate_remaining_days_and_broke
Execute checkSales
Fecha
Last Sale Date
Data successfully inserted into MongoDB!
2023-06-02T00:00:00.000000000
Duplicate rows found:
(12148, 12)
Duplicates resolved. Cleaned data:
(118626, 6)
Execute calculate_l

In [13]:
data['Código Barra CP'] = (data['Código Barra CP']).astype(str)  

In [73]:
df_test = joined_df.copy()
df_test = calculateDF(df_test)

In [75]:
df_test[(df_test['Stock dia actual'] > 0) & (df_test['Venta dia anterior'] > 0)]

,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,Total Presence,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category
2,2023-02-17,1396421761743,AANNGBYHI85,GT,1,8,-1.0,2023-02-17,0.0,176,264,0.666667,Unknown,-2.0,False,8.000000,False,False,False,Producto sano
3,2023-02-18,1396421761743,AANNGBYHI85,GT,2,6,-1.0,2023-02-17,1.0,177,264,0.670455,Unknown,-3.0,False,3.000000,False,True,False,Producto sano
8,2023-02-10,1477119909620,AANNGBYHI85,GT,2,4,-1.0,2023-02-10,0.0,261,264,0.988636,Unknown,-3.0,False,2.000000,False,False,False,Producto sano
9,2023-02-11,1477119909620,AANNGBYHI85,GT,2,20,-1.0,2023-02-10,1.0,259,264,0.981061,Unknown,-3.0,False,10.000000,False,True,False,Producto sano
11,2023-02-18,1477119909620,AANNGBYHI85,GT,4,77,-1.0,2023-02-10,8.0,262,264,0.992424,Unknown,-5.0,False,19.250000,False,True,False,Producto sano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465106,2023-02-18,966867342779,ZZVGGBN1,HN,4,276,-1.0,2023-02-10,8.0,96,114,0.842105,Unknown,-5.0,False,69.000000,False,True,False,Producto sano
465111,2023-02-10,977219179752,ZZVGGBN1,HN,3,35,-1.0,2023-02-10,0.0,111,114,0.973684,Unknown,-4.0,False,11.666667,False,False,False,Producto sano
465112,2023-02-11,977219179752,ZZVGGBN1,HN,5,42,-1.0,2023-02-10,1.0,111,114,0.973684,Unknown,-6.0,False,8.400000,False,True,False,Producto sano
465113,2023-02-17,977219179752,ZZVGGBN1,HN,1,36,-1.0,2023-02-10,7.0,107,114,0.938596,Unknown,-2.0,False,36.000000,False,True,False,Producto sano


#### Procesamiento

In [61]:
def calculateDF(df_test):
    df_test = df_test.sort_values(by=['storeId', 'productId', 'Fecha']).reset_index(drop=True)
    
    df_test = calculate_last_4_weeks_avg_optimized(df_test)    
    print("Execute calculate_last_4_weeks_avg_optimized")
    df_test = calculate_oldest_sale_date_vectorized(df_test)
    print("Execute calculate_oldest_sale_date_vectorized")
    df_test = totalPresence(df_test)
    print("Execute totalPresence")
    df_test = defineRotationType(df_test)
    print("Execute defineRotationType")
    df_test = calculate_percentage_diff_and_condition(df_test)
    print("Execute calculate_percentage_diff_and_condition")
    df_test = calculate_remaining_days_and_broke(df_test)
    print("Execute calculate_remaining_days_and_broke")
    df_test = checkSales(df_test)
    print("Execute checkSales")
    df_test['category'] = df_test.apply(assign_category, axis=1)

    return df_test

### Funciones auxiliares

In [76]:
def preproccingDF(data):
    df_test = data.copy()    
    joined_df = pd.merge(df_test,catalogDescription,how='left',left_on='Código Barra CP',right_on='OriginalBarCode')
    joined_df = pd.merge(joined_df,storeDescription,how='left',left_on='Store Nbr',right_on='OriginalStoreCode')
    joined_df = pd.merge(joined_df,categoryDescription,how='left',left_on='SubBrand CP',right_on='OriginalCategory')
    joined_df = joined_df[['Promedio Rotación Semanal','Inv On Hand','Inv en Tránsito','Inv preparandose en CD','productDescription','productId','storeId','storeDescription','categoryId','País','Fecha','Semana']]
    joined_df = joined_df.rename(columns={"Promedio Rotación Semanal": "Venta dia anterior",
                                "Inv On Hand": "Stock dia actual",
                                "Inv en Tránsito": "Pedido en transito",
                                "Inv preparandose en CD": "Pedido procesandose"                         
                                })
    return cleanDF(joined_df)

def cleanDF(df_test):
    duplicates = df_test.duplicated(subset=['Fecha', 'productId', 'storeId','País'], keep=False)
    
    if duplicates.any():
        print("Duplicate rows found:")
        print(df_test[duplicates].shape)
    
        df_test = (
            df_test.groupby(['Fecha', 'productId', 'storeId','País'], as_index=False)
            .agg({'Venta dia anterior': 'sum', 'Stock dia actual': 'sum'})
        )
        print("Duplicates resolved. Cleaned data:")
        print(df_test.shape)
    return df_test

def calculate_last_4_weeks_avg_optimized(df):
    df['Shifted Venta'] = df.groupby(['storeId', 'productId'])['Venta dia anterior'].shift(1) #Exclude the current date
    df['Rolling Avg'] = (
        df.groupby(['storeId', 'productId'])['Shifted Venta']
        .transform(lambda x: x.rolling(window=4, min_periods=1).mean()) #Average for the 4 previous records
    )
    df['Rolling Count'] = (
        df.groupby(['storeId', 'productId'])['Shifted Venta']
        .transform(lambda x: x.rolling(window=4, min_periods=1).count()) # Count for the rows that has more than 4 records
    )
    
    df['Last 4 Weeks Avg'] = df['Rolling Avg'].where(df['Rolling Count'] >= 4, -1)
    # Drop columns
    df.drop(columns=['Shifted Venta', 'Rolling Avg', 'Rolling Count'], inplace=True)
    return df

def calculate_oldest_sale_date_vectorized(df):
    df = df.sort_values(by=["storeId", "productId", "Fecha"]).reset_index(drop=True)

    sales_mask = df["Venta dia anterior"] > 0

    df["Last Sale Date"] = (
        df[sales_mask]
        .groupby(["storeId", "productId"])["Fecha"]
        .transform(lambda x: x.ffill())
    )
    df["Last Sale Date"] = (
        df.groupby(["storeId", "productId"])["Last Sale Date"]
        .ffill()
    )
    df["Days Since Last Sale"] = (df["Fecha"] - df["Last Sale Date"]).dt.days.fillna(-1).astype(int)

    return df



def totalPresence(df_test):
    df_test['Total Presence'] = (
        df_test[df_test['Stock dia actual'] > 0]
        .groupby(['productId', 'País', 'Fecha'])['storeId']
        .transform('nunique')
        .fillna(0)
        .astype(int)
    )
    
    df_test['Total Presence'] = df_test['Total Presence'].fillna(0).astype(int) # FIll the invalid values
    
    total_stores_per_country = df_test.groupby('País')['storeId'].nunique()
    df_test['total tiendas'] = df_test['País'].map(total_stores_per_country)
    df_test['% global tiendas'] = df_test['Total Presence'] / df_test['total tiendas'] # Calculate the percentage of presence
    
    return df_test

def define_rotation_type(df_test):
    last_dates = (
        df_test.groupby(['storeId', 'productId'])['Fecha']
        .max()
        .reset_index(name='Last Date')
    )
    
    df_test = df_test.merge(last_dates, on=['storeId', 'productId'], how='left')

    last_date_rows = df_test[df_test['Fecha'] == df_test['Last Date']]

    mode_values = (
        last_date_rows.groupby(['storeId', 'productId'])['Days Since Last Sale']
        .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
        .reset_index(name='Mode')
    )
    
    mode_values['Rotation Type'] = mode_values['Mode'].apply(classify_rotation_type)

    df_test = df_test.merge(
        mode_values[['storeId', 'productId', 'Rotation Type']],
        on=['storeId', 'productId'],
        how='left'
    )

    df_test.drop(columns=['Last Date'], inplace=True)

    return df_test


def classify_rotation_type(mode_value):
    if pd.isna(mode_value):
        return 'Unknown'
    elif 0 <= mode_value <= 15:
        return 'A'    
    elif 16 <= mode_value <= 60:
        return 'B'    
    elif mode_value > 60:
        return 'C'    
    else:
        return 'Unknown'


def calculate_percentage_diff_and_condition(df):
    df['Percentage Difference'] = (
        df['Venta dia anterior'] / df['Last 4 Weeks Avg'] - 1
    ).replace([float('inf'), -float('inf')], float('nan'))  # Replace invalid values

    thresholds = {'A': 0.25, 'B': 0.40, 'C': 0.50, 'Unknown': 0.50}
    df['Threshold'] = df['Rotation Type'].map(thresholds)
    df['Condition'] = df['Percentage Difference'] > df['Threshold'].fillna(float('inf')) # Calculate the condition based on percentage of variation of sales and thresholds

    return df

def calculate_remaining_days_and_broke(df):
    df['Remaining Days'] = df['Stock dia actual'] / df['Venta dia anterior'].replace(0, 1)

    thresholds = {
        'A': 4,
        'B': 2,
        'C': 1,
        'Unknown': 1,
    } # Days of availables sale
    
    df['Threshold'] = df['Rotation Type'].map(thresholds).fillna(float('inf'))
    df['Remaining Broke'] = df['Remaining Days'] < df['Threshold'] #Compare if the available days of sales is lower than the day thrseholds
    df.drop(columns=['Threshold'], inplace=True)

    return df


def checkSales(df_test):    
    df_test['cumulative_sales'] = (
        df_test.groupby(['storeId', 'productId'])['Venta dia anterior']
        .cumsum()
        .shift(1, fill_value=0)
    ) # Check if has sale parting of the previous date
    
    df_test['has sales'] = df_test['cumulative_sales'] > 0 # Check if the accumulate is greather than zero
    
    df_test['group_index'] = (
        df_test.groupby(['storeId', 'productId'])
        .cumcount()
    )
    df_test['enough information'] = df_test['group_index'] >= 4 #Check if has more than 4 records of information
    df_test.drop(columns=['cumulative_sales', 'group_index'], inplace=True)
    return df_test

def assign_category(row):
    if row['Stock dia actual'] < 0:
        return "inventario negativo"
    elif not row['has sales'] and row['Venta dia anterior'] == 0:
        return 'producto nuevo sin movimiento'
    elif row['Condition'] and row['Remaining Broke'] and row['Remaining Days'] > 0:
        return "Posible venta atípica"
    elif ((row['Condition'] and row['Remaining Broke']) and row['Remaining Days'] <= 0) or (row['Remaining Broke'] and row['Percentage Difference'] <= 0) :
        return "Posible quiebre de stock por pedido insuficiente"
    elif row['% global tiendas'] <= 0.4:
        return 'Posible producto eliminando de catalogo'
    elif not row['Condition'] and row['Remaining Broke'] and row['Remaining Days'] > 0:
        return 'Pedido insuficiente'
    elif row['Stock dia actual'] == 0:
        if row['Pedido en transito'] > 0:
            return 'Pedido pendiente de entregar'        
        elif row['Pedido procesandose'] > 0:
            return 'Pedido realizado tardiamente'
        else:
            return 'Producto con quiebre de stock'
    elif not row['Remaining Broke']:
        return "Producto sano"
    else:
        return None  # For cases that dont apply a correct assign


### Database Fnctions

In [ ]:
def insertResults(df_test):
    for col in df_test.select_dtypes(include=["datetime64[ns]"]).columns:
        print(col)
        df_test[col] = df_test[col].apply(lambda x: None if pd.isna(x) else x.isoformat() if isinstance(x, pd.Timestamp) else x)

    data_dict = df_test.to_dict(orient="records")
    
    client = MongoClient("mongodb://localhost:27017/")
    db = client["retail_classificator"]
    collection = db["data"]

    if data_dict:  # Only insert if there is data
        collection.insert_many(data_dict)
        print("Data successfully inserted into MongoDB!")
    else:
        print("No data to insert.")

def readResults():
    fields = ['Fecha', 'productId', 'storeId', 'País', 'Venta dia anterior',
       'Stock dia actual', 'Last 4 Weeks Avg', 'Last Sale Date',
       'Days Since Last Sale', 'Total Presence', 'total tiendas',
       #'% global tiendas', 'Rotation Type', 'Percentage Difference'
        '% global tiendas', 'Percentage Difference',
       'Condition', 'Remaining Days', 'Remaining Broke', 'has sales',
       'enough information', 'category']

    client = MongoClient("mongodb://localhost:27017/") 
    db = client["retail_classificator"]
    collection = db["data"]

    data = list(collection.find())
    if len(data) == 0:
        return pd.DataFrame([])

    df_from_db = pd.DataFrame(data)

    datetime_columns = ["Fecha", "Last Sale Date"] 
    for col in datetime_columns:
        if col in df_from_db.columns:
            df_from_db[col] = pd.to_datetime(df_from_db[col], errors="coerce")  
        
    
    print(f"Total of: {df_from_db[fields].shape}")
    return df_from_db[fields]

## Export to training files

In [11]:
from pymongo import MongoClient
import pandas as pd

def export_mongo_to_csv(db_name, collection_name, output_dir, batch_size=300000):
    """
    Export a MongoDB collection to multiple CSV files with a limit of `batch_size` records per file.
    
    Args:
        db_name (str): Name of the MongoDB database.
        collection_name (str): Name of the collection.
        output_dir (str): Directory to store the exported CSV files.
        batch_size (int): Number of records per file.
    """
    # Connect to MongoDB
    client = MongoClient()  # Adjust host and port if needed
    db = client[db_name]
    collection = db[collection_name]

    # Get total count of records in the collection
    total_records = collection.count_documents({})
    print(f"Total records in the collection: {total_records}")

    # Loop through the collection in batches
    for i in range(0, total_records, batch_size):
        # Fetch batch of records using skip and limit
        cursor = collection.find().skip(i).limit(batch_size)
        data = list(cursor)  # Convert cursor to list

        # Convert to pandas DataFrame
        df = pd.DataFrame(data)

        fields = ['Venta dia anterior',
       'Stock dia actual', 'Last 4 Weeks Avg', 
       'Days Since Last Sale', 'Total Presence',
        '% global tiendas', 'Percentage Difference',
       'Condition', 'Remaining Days', 'Remaining Broke', 'has sales',
       'enough information', 'category']

        df=df[fields]
        
        # Drop MongoDB's internal ID field if not needed
        if '_id' in df.columns:
            df.drop('_id', axis=1, inplace=True)
        
        # Define file name and save the file
        file_name = f"{output_dir}/data_part_{i // batch_size + 1}.csv"
        df.to_csv(file_name, index=False)
        print(f"Saved {len(df)} records to {file_name}")

    print("Export completed successfully!")





In [13]:
export_mongo_to_csv(
    db_name="retail_classificator",
    collection_name="data",
    output_dir="data/dummy"
)

Total records in the collection: 4079600
Saved 300000 records to data/dummy/data_part_1.csv
Saved 300000 records to data/dummy/data_part_2.csv
Saved 300000 records to data/dummy/data_part_3.csv
Saved 300000 records to data/dummy/data_part_4.csv
Saved 300000 records to data/dummy/data_part_5.csv
Saved 300000 records to data/dummy/data_part_6.csv
Saved 300000 records to data/dummy/data_part_7.csv
Saved 300000 records to data/dummy/data_part_8.csv
Saved 300000 records to data/dummy/data_part_9.csv
Saved 300000 records to data/dummy/data_part_10.csv
Saved 300000 records to data/dummy/data_part_11.csv
Saved 300000 records to data/dummy/data_part_12.csv
Saved 300000 records to data/dummy/data_part_13.csv
Saved 179600 records to data/dummy/data_part_14.csv
Export completed successfully!


In [39]:
df_test

NameError: name 'df_test' is not defined

In [175]:
df_test[(df_test['Last 4 Weeks Avg'] == -1)]

,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,Total Presence,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category
0,2023-02-10,1396421761743,AANNGBYHI85,GT,0,15,-1.0,NaT,-1.0,177,264,0.670455,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
1,2023-02-11,1396421761743,AANNGBYHI85,GT,0,15,-1.0,NaT,-1.0,173,264,0.655303,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
2,2023-02-17,1396421761743,AANNGBYHI85,GT,1,8,-1.0,2023-02-17,0.0,176,264,0.666667,Unknown,-2.0,False,8.0,False,False,False,Producto sano
3,2023-02-18,1396421761743,AANNGBYHI85,GT,2,6,-1.0,2023-02-17,1.0,177,264,0.670455,Unknown,-3.0,False,3.0,False,True,False,Producto sano
4,2023-02-10,1407709521895,AANNGBYHI85,GT,0,2,-1.0,NaT,-1.0,244,264,0.924242,Unknown,-1.0,False,2.0,False,True,False,Producto sano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465114,2023-02-18,977219179752,ZZVGGBN1,HN,1,35,-1.0,2023-02-10,8.0,106,114,0.929825,Unknown,-2.0,False,35.0,False,True,False,Producto sano
465115,2023-02-10,9988278069237,ZZVGGBN1,HN,0,7,-1.0,NaT,-1.0,111,114,0.973684,Unknown,-1.0,False,7.0,False,True,False,Producto sano
465116,2023-02-11,9988278069237,ZZVGGBN1,HN,0,7,-1.0,NaT,-1.0,111,114,0.973684,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento
465117,2023-02-17,9988278069237,ZZVGGBN1,HN,0,7,-1.0,NaT,-1.0,110,114,0.964912,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento


In [39]:
df_test[(df_test['productId'] == '1396421761743') & (df_test['storeId'] == 'AANNGBYHI85')]

,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,Total Presence,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category
0,2023-02-10,1396421761743,AANNGBYHI85,GT,0,15,-1.0,NaT,-1.0,177,264,0.670455,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
1,2023-02-11,1396421761743,AANNGBYHI85,GT,0,15,-1.0,NaT,-1.0,173,264,0.655303,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
2,2023-02-17,1396421761743,AANNGBYHI85,GT,1,8,-1.0,2023-02-17,0.0,176,264,0.666667,Unknown,-2.0,False,8.0,False,False,False,Producto sano
3,2023-02-18,1396421761743,AANNGBYHI85,GT,2,6,-1.0,2023-02-17,1.0,177,264,0.670455,Unknown,-3.0,False,3.0,False,True,False,Producto sano


In [181]:
df_test[(df_test['productId'] == '9724808238281') & (df_test['País'] == 'CR')]

,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,Total Presence,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category


In [183]:
df_test['productId'] = (df_test['productId']).astype(str)  
df_test.to_excel("finalV2.xlsx",index=False)

In [204]:
# Handling NaT and formatting datetime fields
df_test['Last Sale Date'] = df_test['Last Sale Date'].apply(lambda x: None if pd.isna(x) or x == "NaT" else x)  # Replace NaT with None
df_test['Fecha'] = df_test['Fecha'].apply(lambda x: None if pd.isna(x) else x)  # Replace NaT with None

# (Optional) Convert datetime to ISO 8601 string format if necessary
#df_test['Fecha'] = df_test['Fecha'].dt.strftime('%Y-%m-%dT%H:%M:%S')
#df_test['Last Sale Date'] = df_test['Last Sale Date'].apply(
#    lambda x: x.strftime('%Y-%m-%dT%H:%M:%S') if isinstance(x, pd.Timestamp) else x
#)

df_test

,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,Total Presence,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category
0,2023-02-10T00:00:00,1396421761743,AANNGBYHI85,GT,0,15,-1.0,None,-1.0,177,264,0.670455,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
1,2023-02-11T00:00:00,1396421761743,AANNGBYHI85,GT,0,15,-1.0,None,-1.0,173,264,0.655303,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
2,2023-02-17T00:00:00,1396421761743,AANNGBYHI85,GT,1,8,-1.0,2023-02-17T00:00:00,0.0,176,264,0.666667,Unknown,-2.0,False,8.0,False,False,False,Producto sano
3,2023-02-18T00:00:00,1396421761743,AANNGBYHI85,GT,2,6,-1.0,2023-02-17T00:00:00,1.0,177,264,0.670455,Unknown,-3.0,False,3.0,False,True,False,Producto sano
4,2023-02-10T00:00:00,1407709521895,AANNGBYHI85,GT,0,2,-1.0,None,-1.0,244,264,0.924242,Unknown,-1.0,False,2.0,False,True,False,Producto sano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465114,2023-02-18T00:00:00,977219179752,ZZVGGBN1,HN,1,35,-1.0,2023-02-10T00:00:00,8.0,106,114,0.929825,Unknown,-2.0,False,35.0,False,True,False,Producto sano
465115,2023-02-10T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,111,114,0.973684,Unknown,-1.0,False,7.0,False,True,False,Producto sano
465116,2023-02-11T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,111,114,0.973684,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento
465117,2023-02-17T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,110,114,0.964912,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento


In [208]:
df_test.columns

Index(['Fecha', 'productId', 'storeId', 'País', 'Venta dia anterior',
       'Stock dia actual', 'Last 4 Weeks Avg', 'Last Sale Date',
       'Days Since Last Sale', 'Total Presence', 'total tiendas',
       '% global tiendas', 'Rotation Type', 'Percentage Difference',
       'Condition', 'Remaining Days', 'Remaining Broke', 'has sales',
       'enough information', 'category'],
      dtype='object')

In [206]:
from pymongo import MongoClient

# MongoDB Connection URI (replace with your MongoDB URI)
client = MongoClient("mongodb://localhost:27017/")

# Define the database and collection
db = client["retail_classificator"]  # Replace 'my_database' with your database name
collection = db["data"]  # Replace 'my_collection' with your collection name

# Convert DataFrame to a list of dictionaries
data_dict = df_test.to_dict(orient="records")

# Insert data into MongoDB
collection.insert_many(data_dict)

print("Data successfully inserted into MongoDB!")


Data successfully inserted into MongoDB!


In [214]:
fields = ['Fecha', 'productId', 'storeId', 'País', 'Venta dia anterior',
       'Stock dia actual', 'Last 4 Weeks Avg', 'Last Sale Date',
       'Days Since Last Sale', 'Total Presence', 'total tiendas',
       '% global tiendas', 'Rotation Type', 'Percentage Difference',
       'Condition', 'Remaining Days', 'Remaining Broke', 'has sales',
       'enough information', 'category']

fields_to_retrieve = {field: 1 for field in fields}
fields_to_retrieve["_id"] = 0  # Exclude the MongoDB '_id' field

# Output the resulting dictionary
print(fields_to_retrieve)

{'Fecha': 1, 'productId': 1, 'storeId': 1, 'País': 1, 'Venta dia anterior': 1, 'Stock dia actual': 1, 'Last 4 Weeks Avg': 1, 'Last Sale Date': 1, 'Days Since Last Sale': 1, 'Total Presence': 1, 'total tiendas': 1, '% global tiendas': 1, 'Rotation Type': 1, 'Percentage Difference': 1, 'Condition': 1, 'Remaining Days': 1, 'Remaining Broke': 1, 'has sales': 1, 'enough information': 1, 'category': 1, '_id': 0}


In [234]:
# MongoDB connection setup
client = MongoClient("mongodb://localhost:27017/")
db = client["retail_classificator"]  # Replace with your database name
collection = db["data"]  # Replace with your collection name

data = list(collection.find())

print(data)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [242]:
df_from_db = pd.DataFrame(data)

In [244]:
df_from_db

,_id,Fecha,productId,storeId,País,Venta dia anterior,Stock dia actual,Last 4 Weeks Avg,Last Sale Date,Days Since Last Sale,...,total tiendas,% global tiendas,Rotation Type,Percentage Difference,Condition,Remaining Days,Remaining Broke,has sales,enough information,category
0,67493ae7c224814e5fcc1d31,2023-02-10T00:00:00,1396421761743,AANNGBYHI85,GT,0,15,-1.0,None,-1.0,...,264,0.670455,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
1,67493ae7c224814e5fcc1d32,2023-02-11T00:00:00,1396421761743,AANNGBYHI85,GT,0,15,-1.0,None,-1.0,...,264,0.655303,Unknown,-1.0,False,15.0,False,False,False,producto nuevo sin movimiento
2,67493ae7c224814e5fcc1d33,2023-02-17T00:00:00,1396421761743,AANNGBYHI85,GT,1,8,-1.0,2023-02-17T00:00:00,0.0,...,264,0.666667,Unknown,-2.0,False,8.0,False,False,False,Producto sano
3,67493ae7c224814e5fcc1d34,2023-02-18T00:00:00,1396421761743,AANNGBYHI85,GT,2,6,-1.0,2023-02-17T00:00:00,1.0,...,264,0.670455,Unknown,-3.0,False,3.0,False,True,False,Producto sano
4,67493ae7c224814e5fcc1d35,2023-02-10T00:00:00,1407709521895,AANNGBYHI85,GT,0,2,-1.0,None,-1.0,...,264,0.924242,Unknown,-1.0,False,2.0,False,True,False,Producto sano
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465114,67493aecc224814e5fd3360b,2023-02-18T00:00:00,977219179752,ZZVGGBN1,HN,1,35,-1.0,2023-02-10T00:00:00,8.0,...,114,0.929825,Unknown,-2.0,False,35.0,False,True,False,Producto sano
465115,67493aecc224814e5fd3360c,2023-02-10T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,...,114,0.973684,Unknown,-1.0,False,7.0,False,True,False,Producto sano
465116,67493aecc224814e5fd3360d,2023-02-11T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,...,114,0.973684,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento
465117,67493aecc224814e5fd3360e,2023-02-17T00:00:00,9988278069237,ZZVGGBN1,HN,0,7,-1.0,None,-1.0,...,114,0.964912,Unknown,-1.0,False,7.0,False,False,False,producto nuevo sin movimiento
